In [2]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
from fuzzywuzzy import fuzz
from ast import literal_eval
from surprise import SVD, SVDpp, KNNBasic
from surprise import Dataset
from surprise.model_selection import cross_validate,train_test_split, GridSearchCV
from surprise import NormalPredictor
from surprise import Reader
from surprise import accuracy

import re 
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import nltk

nltk.download('stopwords')

/opt/conda/lib/python3.7/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/jupyter/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate
import lightgbm as lgb
import warnings; warnings.simplefilter('ignore')

In [4]:
1

1

In [5]:
train_features = pd.read_csv('train_features.csv')
test_features = pd.read_csv('test_features.csv')

In [6]:
pd.options.display.max_columns = None
pd.options.display.max_rows = None

np.set_printoptions(threshold=np.inf)

In [7]:
train_features.drop('song_length_y',axis = 1 , inplace =True)
train_features.drop('genre_ids_y',axis = 1 , inplace =True)
train_features.drop('artist_name_y',axis = 1 , inplace =True)
train_features.drop('composer_y',axis = 1 , inplace =True)
train_features.drop('lyricist_y',axis = 1 , inplace =True)
train_features.drop('language_y',axis = 1 , inplace =True)

test_features.drop('song_length_y',axis = 1 , inplace =True)
test_features.drop('genre_ids_y',axis = 1 , inplace =True)
test_features.drop('artist_name_y',axis = 1 , inplace =True)
test_features.drop('composer_y',axis = 1 , inplace =True)
test_features.drop('lyricist_y',axis = 1 , inplace =True)
test_features.drop('language_y',axis = 1 , inplace =True)

In [8]:
train_features.drop('lyricist_x',axis = 1 , inplace =True)
test_features.drop('lyricist_x',axis = 1 , inplace =True)

train_features.drop('source_type',axis = 1 , inplace =True)
test_features.drop('source_type',axis = 1 , inplace =True)



In [9]:
def keepwords(text):
    temp = re.sub('[^a-zA-Z]', ' ', str(text))
    return temp.lower() + ' '

#train_features['name'] = train_features['name'].astype('category')
train_features['name'] = train_features.name.apply(lambda x: keepwords(x))
test_features['name'] = test_features.name.apply(lambda x: keepwords(x))

#train_features['artist_name_x'] = train_features['artist_name_x'].astype('category')
train_features['artist_name_x'] = train_features.artist_name_x.apply(lambda x: keepwords(x))
test_features['artist_name_x'] = test_features.artist_name_x.apply(lambda x: keepwords(x))

#train_features['source_screen_name'] = train_features['source_screen_name'].astype('category')
train_features['source_screen_name'] = train_features.source_screen_name.apply(lambda x: keepwords(x))
test_features['source_screen_name'] = test_features.source_screen_name.apply(lambda x: keepwords(x))

train_features['composer_x'] = train_features.composer_x.apply(lambda x: keepwords(x))
test_features['composer_x'] = test_features.composer_x.apply(lambda x: keepwords(x))



In [10]:
1

1

In [11]:
train_features['source_system_tab'] = train_features['source_system_tab'].fillna('')
train_features['source_screen_name'] = train_features['source_screen_name'].fillna('')
#train_features['source_type'] = train_features['source_type'].fillna('')
train_features['song_length_x'] = train_features['song_length_x'].fillna(0)
train_features['genre_ids_x'] = train_features['genre_ids_x'].fillna(0)
train_features['artist_name_x'] = train_features['artist_name_x'].fillna('')
#train_features['lyricist_x'] = train_features['lyricist_x'].fillna('')
train_features['composer_x'] = train_features['composer_x'].fillna('')
train_features['language_x'] = train_features['language_x'].fillna(0)
train_features['name'] = train_features['name'].fillna('')
train_features['isrc'] = train_features['isrc'].fillna(0)
train_features['city'] = train_features['city'].fillna(0)
train_features['bd'] = train_features['bd'].fillna(0)
train_features['gender'] = train_features['gender'].fillna('')
train_features['registered_via'] = train_features['registered_via'].fillna(0)
train_features['registration_init_time'] = train_features['registration_init_time'].fillna('1901-01-01')
train_features['expiration_date'] = train_features['expiration_date'].fillna('1901-01-01')
train_features['repeat_count'] = train_features['repeat_count'].fillna(0)
train_features['play_count'] = train_features['play_count'].fillna(0)
train_features['repeat_percentage'] = train_features['repeat_percentage'].fillna(0)
train_features['repeat_count_artist'] = train_features['repeat_count_artist'].fillna(0)
train_features['play_count_artist'] = train_features['play_count_artist'].fillna(0)
train_features['repeat_count_msno'] = train_features['repeat_count_msno'].fillna(0)
train_features['play_count_msno'] = train_features['play_count_msno'].fillna(0)
train_features['repeat_percentage_msno'] = train_features['repeat_percentage_msno'].fillna(0)
train_features['song_year'] = train_features['song_year'].fillna(0)
#train_features['description'] = train_features['description'].fillna(0)




In [12]:
train_features['description'] = train_features['name'] + train_features['artist_name_x']
test_features['description'] = test_features['name'] + test_features['artist_name_x']
train_features['description'] = train_features['description'].fillna('')
tf = TfidfVectorizer(analyzer='word', ngram_range=(1,2), min_df=0, stop_words='english')

In [13]:
TFidvectorizer = TfidfVectorizer(max_df=0.5, max_features=10000,
                                 min_df=2, stop_words='english',
                                 use_idf=True)
tf_vector = TfidfVectorizer()
transformer = TfidfTransformer()

In [14]:
#tfidf = transformer.fit_transform(tf_vector.fit_transform(train_features.description))
tfidf = transformer.fit_transform(tf_vector.fit_transform(test_features.description))


In [17]:
#cosine_sim = cosine_similarity(tfidf, tfidf)

In [15]:
tfidf.shape

(837941, 32348)

In [16]:
import gc
gc.collect()

44

In [17]:
chunk_size = 500
matrix_len = tfidf.shape[0] # Not sparse numpy.ndarray

def similarity_cosine_by_chunk(start, end):
    if end > matrix_len:
        end = matrix_len
    return cosine_similarity(X=tfidf[start:end], Y=tfidf[start:end]) # scikit-learn function



In [18]:
cosine_sim = np.array([])  # to keep the final result of similarities
cosine_sim_list = []
for chunk_start in range(0, matrix_len, chunk_size):
    if cosine_sim.shape[0] % 20000 == 0:
        print(cosine_sim.shape)
    cosine_similarity_chunk = similarity_cosine_by_chunk(chunk_start, chunk_start + chunk_size)
    if cosine_sim.shape[0] == 0:
        cosine_sim =  np.copy(cosine_similarity_chunk)
        #cosine_sim_list.append(cosine_similarity_chunk)
        del cosine_similarity_chunk
    else:
        cosine_sim =  np.concatenate((cosine_sim,cosine_similarity_chunk), axis=0)
        #cosine_sim_list.append(cosine_similarity_chunk)
        del cosine_similarity_chunk


(0,)
(20000, 500)
(40000, 500)
(60000, 500)
(80000, 500)
(100000, 500)
(120000, 500)
(140000, 500)
(160000, 500)
(180000, 500)
(200000, 500)
(220000, 500)
(240000, 500)
(260000, 500)
(280000, 500)
(300000, 500)
(320000, 500)
(340000, 500)
(360000, 500)
(380000, 500)
(400000, 500)
(420000, 500)
(440000, 500)
(460000, 500)
(480000, 500)
(500000, 500)
(520000, 500)
(540000, 500)
(560000, 500)
(580000, 500)
(600000, 500)
(620000, 500)
(640000, 500)
(660000, 500)
(680000, 500)
(700000, 500)
(720000, 500)
(740000, 500)
(760000, 500)
(780000, 500)
(800000, 500)
(820000, 500)


ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 500 and the array at index 1 has size 441

In [186]:
np.savetxt("cosine_sim.txt", cosine_sim,fmt='%f',delimiter=',')
print(cosine_sim.shape)
len(cosine_sim)
cosine_sim[1][499]

(837500, 500)


0.0

In [187]:
asd = np.loadtxt('cosine_sim.txt',delimiter=',')
asd.shape

KeyboardInterrupt: 

In [24]:
# train_features = train_features.reset_index()
# titles = train_features['name']
# indices = pd.Series(train_features.index, index = train_features['description'])

test_features = test_features.reset_index()
titles = test_features['name']
indices = pd.Series(test_features.index, index = test_features['description'])

In [40]:
def get_index_by_name(name):
    specific_songs = test_features.loc[test_features.name.str.contains(name)]
    #specific_songs = test_features.loc[test_features.name.str.contains(name)]
    dd = specific_songs.head(5)
    #print(dd.index)
    return dd.index

def get_scores(slist):
    res = []
    for i in slist:
        if i != 0:
            res.append(i)
    return res

def get_cosine_sim_scores2(idxs):
    res = []
    for i in idxs:
        sim_scores = list(enumerate(cosine_sim[i]))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        sim_scores = sim_scores[1:31]
        movie_indices = [i[0] for i in sim_scores]
        temp = titles.iloc[movie_indices]
        res.append(temp)

    return res

def get_cosine_sim_scores(idx):
    res = []
    for i in range(len(cosine_sim_list)):
        if len(consine_sim_list[i]) > idx:
            temp = cosine_sim_list[i][idx]
            res.append(temp)
            
        #temp = map(lambda x: get_scores(x), cosine_sim_list)
    sim_scores = list(enumerate(res))
    #print(sim_scores)
    return sim_scores

def get_recommendation(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:26]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]    
    
        
def get_recommendations2(title):
    idxs = get_index_by_name(title)    
    sim_scores = get_cosine_sim_scores2(idxs)
    return sim_scores

In [41]:
recom = get_recommendations2('my love')

In [44]:
recom = get_recommendations2('my love')

In [64]:
def remove_blank(recom):
    res = dict()
    for r in recom:
        for i, v in r.items():
            temp = ' '.join(v.split())
            if temp != '':
                res[i] = temp
    return res

In [65]:
remove_blank(recom)

{214: 'my only love',
 308: 'play',
 355: 'play',
 105: 'my way to love',
 156: 'be my lady',
 183: 'love u u',
 154: 'on my way',
 346: 'on my way',
 436: 'you are my mr right',
 13: 'my dad',
 274: 'be my own friend',
 360: 'beautiful love beautiful love',
 177: 'my love',
 178: 'good night',
 179: 'while my guitar gently weeps',
 180: 'a man without love quando m innamoro',
 115: 'my love',
 334: 'this love',
 414: 'you exist in my song',
 332: 'unbreakable love',
 283: 'practice love',
 207: 'dancing on my own',
 402: 'my boo',
 426: 'love somebody',
 190: 'thanks for your love',
 314: 'i stay in love',
 138: 'why not love',
 171: 'love affair',
 345: 'love on the weekend',
 195: 'how deep is your love',
 43: 'feat e so who we love',
 293: 'what love has taught us',
 313: 'what love has taught us',
 383: 'crests of waves',
 89: 'run up feat partynextdoor nicki minaj',
 444: 'take me to church',
 167: 'nini',
 349: 'crush',
 292: 'i loved you',
 202: 'i really like you',
 29: 'stuck

In [46]:
algo = SVD()
reader = Reader(rating_scale=(0,1))
data = Dataset.load_from_df(train_features[['msno', 'song_id', 'target']].dropna(), reader)
trainset, testset = train_test_split(data, test_size=.25)
trainset = data.build_full_trainset()


In [47]:
# Train the algorithm on the trainset, and predict ratings for the testset
algo.fit(trainset)

In [48]:
1

1

In [49]:
predictions = algo.test(testset)

# Then compute RMSE
accuracy.rmse(predictions)

RMSE: 0.4274


0.4274175256264111

In [50]:
# submit = []
# for index, row in test_features.iterrows():
#     est = algo.predict(row['msno'], row['song_id']).est
#     submit.append((row['id'], est))

#est = algo.predict(row['msno'], row['song_id']).est

In [40]:
test_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 837941 entries, 0 to 837940
Data columns (total 29 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   index                   837941 non-null  int64  
 1   id                      837941 non-null  int64  
 2   msno                    837941 non-null  object 
 3   song_id                 837941 non-null  object 
 4   source_system_tab       835145 non-null  object 
 5   source_screen_name      837941 non-null  object 
 6   song_length_x           837934 non-null  float64
 7   genre_ids_x             826839 non-null  object 
 8   artist_name_x           837941 non-null  object 
 9   composer_x              837941 non-null  object 
 10  language_x              837925 non-null  float64
 11  name                    837941 non-null  object 
 12  isrc                    772937 non-null  object 
 13  city                    837941 non-null  int64  
 14  bd                  

In [32]:
drop_list = ['repeat_count','repeat_percentage','repeat_count_msno','repeat_percentage_msno']
drop_list2 = ['Unnamed: 0', 'Unnamed: 0.1']
train_features = train_features.drop(drop_list2,axis=1)
test_features = test_features.drop(drop_list2,axis=1)

In [33]:
for col in train_features.columns:
    if train_features[col].dtype == object:
        train_features[col] = train_features[col].astype('category')
        test_features[col] = test_features[col].astype('category')

In [36]:
sample = test_features.head(10)

In [123]:
gbm = lgb.Booster(model_file='model.txt')

In [156]:
def hybrid(userId, title):
    
    idxs = get_index_by_name(title)    
    sim_scores = get_cosine_sim_scores2(idxs)
    movies = remove_blank(sim_scores)
    movies = getFrame_by_indexs(movies)
    

    movies = movies.drop(['id','index'], axis=1)
    movies['msno'] = userId
    movies = change_data_type(movies)
    
    movies['est'] = gbm.predict(movies)
    #movies['est'] = movies['song_id'].apply(lambda x: algo.predict(userId, x).est)
                                 
#     movies = movies.sort_values('est', ascending=False)
#     return movies.head(10)
    return movies

def change_data_type(music):
    drop_list2 = ['Unnamed: 0', 'Unnamed: 0.1']
    music = music.drop(drop_list2,axis=1)
    for col in music.columns:
        if music[col].dtype == object:
            #test_features[col] = train_features[col].astype('category')
            music[col] = music[col].astype('category')
    return music

def getFrame_by_indexs(movies):
    indexs = []
    for i, v in movies.items():
        indexs.append(i)
    indexs = sorted(indexs)
    begin = test_features.iloc[indexs[-1]:indexs[-1]+1]
    for i in range(len(indexs)-1):
        results = test_features.iloc[indexs[i]:indexs[i]+1]
        begin = pd.concat([begin, results])

    return begin


In [157]:
movies = hybrid("91GgOFj9CDFYxxqRxQBT5L7FYjborQ40TReKAqpfQrs=",'my love')

In [162]:
recom = movies[['msno', 'song_id', 'source_system_tab', 'source_screen_name', 'name', 'play_count', 'est']]
recom = recom.sort_values('est', ascending=False)

In [165]:
recom.head(20)

,msno,song_id,source_system_tab,source_screen_name,name,play_count,est
257,91GgOFj9CDFYxxqRxQBT5L7FYjborQ40TReKAqpfQrs=,QIGvqbdLL7FD0+VJSKoTSl4WMdgNzgs2wC6x823R4C0=,my library,local playlist more,the best is yet to come,1.0,0.999481
324,91GgOFj9CDFYxxqRxQBT5L7FYjborQ40TReKAqpfQrs=,vZRVbOd3WV+Yt6LqwVcxqYI4JQGah/cW4r6uWumi/5E=,my library,local playlist more,on,159.0,0.997910
283,91GgOFj9CDFYxxqRxQBT5L7FYjborQ40TReKAqpfQrs=,n+pMhj/jpCnpiUcSDl4k3i9FJODDddEXmpE48/HczTI=,discover,online playlist more,practice love,8775.0,0.986623
43,91GgOFj9CDFYxxqRxQBT5L7FYjborQ40TReKAqpfQrs=,CU7wosdFGNmijUjuXNisWoD6XGpkFZHAVD78E5ImVX4=,my library,local playlist more,feat e so who we love,527.0,0.974298
190,91GgOFj9CDFYxxqRxQBT5L7FYjborQ40TReKAqpfQrs=,PgRtmmESVNtWjoZHO5a1r21vIz9sVZmcJJpFCbRa1LI=,my library,local playlist more,thanks for your love,12004.0,0.969479
313,91GgOFj9CDFYxxqRxQBT5L7FYjborQ40TReKAqpfQrs=,bTlSHgnwaBMIKHseTFHanJ3Zr4x82ybrGkexpnxDhTY=,my library,local playlist more,what love has taught us,5508.0,0.941761
426,91GgOFj9CDFYxxqRxQBT5L7FYjborQ40TReKAqpfQrs=,0ftZO/ByXbLHmtLPt5NwATkT3OrbCuZ7CvrsnL3BvqQ=,my library,local playlist more,love somebody,527.0,0.939822
494,91GgOFj9CDFYxxqRxQBT5L7FYjborQ40TReKAqpfQrs=,J4qKkLIoW7aYACuTupHLAPZYmRp08en1AEux+GSUzdw=,my library,local playlist more,faded,10565.0,0.938185
183,91GgOFj9CDFYxxqRxQBT5L7FYjborQ40TReKAqpfQrs=,lpehYTGIoMuZRbdiEiXEBJbsZ/UhTV0XYxE1EOablJA=,my library,local playlist more,love u u,1983.0,0.928002
202,91GgOFj9CDFYxxqRxQBT5L7FYjborQ40TReKAqpfQrs=,NO3PiEzofJglKjD8+biTGWkEVRDfE799y8c9wQS1XWo=,my library,local playlist more,i really like you,4062.0,0.923077
